## Calculate burst-glide statistics and lower-frequency speed changes for each fish in a trial

### Using speed data for each fish:
   - identify time windows without tracking errors/gaps
   - identify burst timing and features (max speed, peak-valley amplitude, rise time, fall time, potentially fit parameters)
   - get sliding burst rate and smoothed speed over time for analyzing speed crosscorrelation in lower frequencies
    

In [1]:
#import needed modules
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd  
import glob
import get_kinematics_adult as kin

In [2]:
# define directories and import list of lines
datadir = '../data/'
resultdir = 'savedresults/'

[treatments] = pickle.load(open(resultdir+'treatmentlist.pkl','rb'))
treatments = np.array(treatments)
numtreatments = len(treatments)
[focustreatments,notfocus] = pickle.load(open(resultdir+'focustreatmentlist.pkl','rb'))
[allnumtrials] = pickle.load(open(resultdir+'allnumtrials.pkl','rb'))

pxpercm = 4.02361434 * 10  # from tracker
dt=1/60

In [3]:
#import single line
tnum = 0  # select which line, 0 = WT
treatment = treatments[tnum]
numtrials = allnumtrials[tnum]
[trial_speeds,trial_trajectories,trial_headings,trial_theta,
            trial_smoothspeeds,trial_smoothtrajectories,trial_smoothheadings,
            trial_ellipses,trial_arena,trial_sex,
            datafiles,trial_trackingerrors] = pickle.load(open(datadir+treatment+'-alltrials.pkl','rb'))

In [4]:
#load fishpair distances from file
filename = datadir+treatment+'-dcoords+dist-heading.pkl'
[trial_dcoords,trial_dist] = pickle.load(open(filename,'rb'))

In [5]:
#trial_headings[0]

In [6]:
#select single trial for line and fish to include in analysis 
#(later: setup to iterate through all trials & save relevant data)
trialnum = 0
fishIncl = np.arange(6)
nframes = trial_trajectories[trialnum].shape[0]

## Identify frames with bad data for each fish

In [7]:
#initialize badData dataframe; types: nan, flatspeed, doubleID (note which), other
errType = pd.DataFrame(np.zeros([nframes,fishIncl.size], dtype=int), columns = ['fish0','fish1','fish2','fish3','fish4','fish5'])
errType.index.name = 'frame'
errType.head()
errCode = {0: 'noError',
 1: 'missing',
 2: 'flatSpeed',
 30: 'doubleID_0',
 31: 'doubleID_1',
 32: 'doubleID_2',
 33: 'doubleID_3',
 34: 'doubleID_4',
 35: 'doubleID_5'}

In [8]:
#add nans/missing data to errType
errs = trial_trackingerrors[trialnum][:,fishIncl].copy()
errType[errs] = 1

In [9]:
#find flatspeed for each fish - speed does not change (or change < threshold) for >t duration; define as baddata
speeds = trial_speeds[trialnum].copy()
accel = np.gradient(speeds,axis=0)
accel[1:] = speeds[1:]-speeds[0:-1]
accel[np.isnan(accel)] = 1.0 #set nans to real value > threshold
errs = (np.abs(accel)<0.001)
errType[errs] = 2

In [10]:
#kin.getBGcycle(speeds[:,0]) #need to do something about nans or mod kin to deal with them before this will work

In [11]:
#define baddata for at least one fish in pair when [dist,oridiff] < threshold (doubleID error)

#for each fishpair, each time point, get distance (trial_dist from file) and orientation diff (subtr heading)
dthresh = pxpercm #1 cm, ~40 pixels; same threshold used in paper; I may use looser threshold
othresh = .2 #taken from paper

for i in fishIncl:
    col0 = errType.columns[i]
    errcode1 = 30 + i
    for j in range(i+1,fishIncl.size):
        col1 = errType.columns[j]
        errcode0 = 30 +j
        dpair = trial_dist[trialnum][:,i,j].copy() 
        dpair[np.isnan(dpair)] = 100 #set nans to high values to avoid invalid values and false errID
        oridiff = trial_headings[trialnum][:,i]-trial_headings[trialnum][:,j]
        oridiff[np.isnan(oridiff)] = 6 #set nans to high values to avoid invalid values and false errID
        indErr = (abs(oridiff) < othresh) & (dpair < dthresh)
        errType.loc[indErr,col0] = errcode0
        errType.loc[indErr,col1] = errcode1

In [17]:
# restore speed and position for nans during bursts (were filtered by accel filter in orig data processing)
timeTH = 25 #preliminary guess for threshold
speedTH = 500 #preliminary guess for threshold
errcode = 8

groupdatadir = datadir + '/raw/' + treatment +'/'
datafiles = np.sort([str.split(s,'/')[-1] for s in glob.glob(groupdatadir+'*mat')])

#open datafile (one for now, set to iterate through after tests); get trajectories, speed, heading
data = loadmat(datafiles[trialnum])
def loadval(dictval):
    return np.array([t[0][0] for t in (np.array(data['trx'][dictval]).T)]) 
data_x = loadval('x')
data_y = loadval('y')
trajectories_raw = np.array([data_x,data_y]).T
velocities = np.gradient(trajectories,axis=0)/dt #if these are too slow, could do just for regions of interest
speeds = np.sqrt(velocities[:,:,0]**2 + velocities[:,:,1]**2) 
headings = np.arctan2(velocities[:,:,1],velocities[:,:,0]) 

#for fish in fishIncl:
    #find regions of interest: criteria: NaN value, duration < fthresh, speedmax - speedmin > speedTH
        #to do, make array: for each errType == 1, col1 : last errType == 0, col2: next errType == 0; 
        #err1bouts = unique rows in array
        #for each err1bout, get: speedchange (after-before), bouttime; 
        #if speedchange > speedTH & bouttime < timeTH, add bout to regions
    #for region in regions:
        #assign new errortype for region
        #replace trajectories w raw 
        #replace speed 
        #compute ori for region
        #maybe compute other params (draw from "data import and smoothing" code)

#??resave pickle? or new pickle? undecided, could also keep local if not too slow


'WT\\data09092015_10_26_AMAB.mat'

In [13]:
# plots to validate errtypes, burstgap fixes, overall data quality

In [14]:
# maybe look for small sharp oscillations? hard to define though, skip for now and maybe come back later

In [15]:
# maybe: kernel or wavelet smooth; don't want to do for now (I think burst coordination analysis may be better w/o it) but consider for later

## find bursts in speed trajectory

   - Build on get_kinematics function for getting BGCycle
   - Tweak parameters for burst detection (especially nback)
   - Maybe: try w both smoothed and raw trajectories, see which is better
   - Either only do for good trajectories, or note when burst is baddata